In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from matplotlib.colors import ListedColormap
import random
import warnings

warnings.filterwarnings('ignore')

### Загружаем датасет Ирис и разбиваем его на подвыборки.

In [2]:
from sklearn import datasets
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)

In [3]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=1)

### Добавляем стандартизацию данных:

In [4]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

### Тестируем на алгоритме KNN: 

In [6]:
def e_metrics(x1, x2):
    distance = 0
    for i in range(len(x1)):
        distance += np.square(x1[i] - x2[i])
        
    return np.sqrt(distance)

def knn(x_train, y_train, x_test, k):
    
    answers = []
    for x in x_test:
        test_distances = []
        
        for i in range(len(x_train)):
            distance = e_metrics(x, x_train[i])
            test_distances.append((distance, y_train[i]))
        
        classes = {class_item: 0 for class_item in set(y_train)}
        
        for d in sorted(test_distances)[0:k]:
            classes[d[1]] += 1
            
        answers.append(sorted(classes, key=classes.get)[-1])
        
    return answers

def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [7]:
k = 5
y_pred = knn(X_train_std, y_train, X_test_std, k)
print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 5: 0.974


### Также загрузим DecisionTreeClassifier из sklearn:

In [8]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=1)
tree.fit(X_train_std, y_train)

DecisionTreeClassifier(max_depth=4, random_state=1)

In [9]:
from sklearn.metrics import classification_report

y_pred_tree = tree.predict(X_test_std)
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_test, y_pred_tree, target_names=target_names))

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00        13
     class 1       1.00      0.94      0.97        16
     class 2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



### Как мы видим, алгоритмы показали почти идеальные результаты. Подправим гиперпараметры, чтобы ухудшить результат перед применением PCA:

In [10]:
k = 70
y_pred = knn(X_train_std, y_train, X_test_std, k)
print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 70: 0.763


In [11]:
tree = DecisionTreeClassifier(criterion='gini', max_depth=1, random_state=1)
tree.fit(X_train_std, y_train)
y_pred_tree = tree.predict(X_test_std)
print(classification_report(y_test, y_pred_tree, target_names=target_names))

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00        13
     class 1       0.00      0.00      0.00        16
     class 2       0.36      1.00      0.53         9

    accuracy                           0.58        38
   macro avg       0.45      0.67      0.51        38
weighted avg       0.43      0.58      0.47        38



### Добавим PCA:

In [12]:
X_ = X.astype(float)
rows, cols = X_.shape
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]
        
covariance_matrix = X_.T.dot(X_)
eig_values, eig_vectors = np.linalg.eig(covariance_matrix)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]
eig_pairs.sort(key=lambda x: x[0], reverse=True)        
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))
Z = X_.dot(W)

In [13]:
X.shape, Z.shape

((150, 4), (150, 2))

### Как видно из размерности, PCA убрал два признака.

### Снова разобъем данные на выборки и посчитаем результат на двух моделях:

In [14]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(Z, y, test_size=0.25, random_state=1)

In [15]:
k = 70
y_pred = knn(X_train, y_train, X_test, k)
print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 70: 0.737


In [16]:
tree = DecisionTreeClassifier(criterion='gini', max_depth=1, random_state=1)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)
print(classification_report(y_test, y_pred_tree, target_names=target_names))

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00        13
     class 1       0.00      0.00      0.00        16
     class 2       0.36      1.00      0.53         9

    accuracy                           0.58        38
   macro avg       0.45      0.67      0.51        38
weighted avg       0.43      0.58      0.47        38



### Как можно заметить, результат KNN изменился незначительно, а результат дерева не изменился вообще. Для данного датасета PCA не смог повысить результаты, но, убрав два "лишних" признака, если бы данных было больше, возможно смог бы сократить временные затраты на обучение и предсказание.